In [26]:
import { create, insertMultiple, insert, search } from 'npm:@orama/orama'
import { restoreFromFile, persistToFile } from 'npm:@orama/plugin-data-persistence/server'
import { pipeline, env } from "https://cdn.jsdelivr.net/npm/@xenova/transformers";
env.backends.onnx.wasm.numThreads = 1;

const embedder = await pipeline("feature-extraction", "Xenova/all-MiniLM-L6-v2")

const embedderOptions = { pooling: "mean", normalize: true}

1

In [2]:
const issues = JSON.parse(Deno.readTextFileSync("../api/issues-cache.json"))

In [15]:
const vectorDB = await create({
    schema: {
        title: "string",
        body: "string",
        url: "string",
        id: "string",
        vector: "vector[384]",
    }
})

In [10]:
const vectors = []

for (const issue of issues) {
    const em = await embedder(issue.title + ": " + issue.body, {
      pooling: "mean",
      normalize: true,
    });
    vectors.push({
        id: issue.id,
        vector: em,
        url: issue.html_url,
        title: issue.title,
        body: issue.body,
    });
}

2447

In [22]:
const vecsToList = vectors.map(v=>{
    return {
        vector: v.vector.tolist().flat(),
        id: v.id.toString(),
        title: v.title,
        body: v?.body ?? "",
        url: v?.url ?? ""
    }
})

In [19]:
const vecsFloats = vectors.map(v=>{
    return {
        vector: v.vector.data,
        id: v.id.toString(),
        title: v.title,
        body: v?.body ?? "",
        url: v?.url ?? ""
    }
})

In [24]:
await insertMultiple(vectorDB, vecsToList)

Error: A document with id "2318612341" already exists.

In [28]:
await persistToFile(vectorDB,"binary", "../api/issues-embeddings.msp")

"../api/issues-embeddings.msp"

In [13]:
const logs = await (await fetch("http://localhost:8788/v0/logs")).json();

const jsxSampleError = logs.logs[2].message.message
jsxSampleError

"process is not defined"

In [14]:
const jsxSampleErrorEmbedding = (await embedder(jsxSampleError, embedderOptions)).data

In [ ]:
vectorDB

{
  data: {
    index: {
      sharedInternalDocumentStore: {
        idToInternalId: Map(2447) {
          "77922575-1" => 1,
          "77922575-2" => 2,
          "77922575-3" => 3,
          "77922575-4" => 4,
          "77922575-5" => 5,
          "77922575-6" => 6,
          "77922575-7" => 7,
          "77922575-8" => 8,
          "77922575-9" => 9,
          "77922575-10" => 10,
          "77922575-11" => 11,
          "77922575-12" => 12,
          "77922575-13" => 13,
          "77922575-14" => 14,
          "77922575-15" => 15,
          "77922575-16" => 16,
          "77922575-17" => 17,
          "77922575-18" => 18,
          "77922575-19" => 19,
          "77922575-20" => 20,
          "77922575-21" => 21,
          "77922575-22" => 22,
          "77922575-23" => 23,
          "77922575-24" => 24,
          "77922575-25" => 25,
          "77922575-26" => 26,
          "77922575-27" => 27,
          "77922575-28" => 28,
          "77922575-29" => 29,
          "77922575-3

In [26]:
const results = await search(vectorDB, {
  mode: 'vector',
  vector: {
    value: jsxSampleErrorEmbedding,
    property: 'embedding',
  },
  similarity: 0.3,      // Minimum similarity. Defaults to `0.8`
  limit: 3,             // Defaults to `10`
  offset: 0,             // Defaults to `0`
})

In [27]:
results

{
  count: 24,
  hits: [
    {
      id: "77922575-1763",
      score: 0.48561719339137566,
      document: {
        embedding: null,
        title: "`c.runtime === 'node'` does not work well",
        url: "https://github.com/honojs/hono/issues/685",
        body: "`c.runtime === 'node'` does not work well. It is because the `global.process.title` is such as`/User"... 97 more characters
      }
    },
    {
      id: "77922575-456",
      score: 0.376411462347048,
      document: {
        embedding: null,
        title: "Can't Run Cloudflare Pages Template On Fresh Initialization on Windows",
        url: "https://github.com/honojs/hono/issues/2182",
        body: "### What version of Hono are you using?\r\n" +
          "\r\n" +
          "4.0\r\n" +
          "\r\n" +
          "### What runtime/platform is your app running on?\r"... 1916 more characters
      }
    },
    {
      id: "77922575-2246",
      score: 0.35996850925483187,
      document: {
        embedding: null,
   